# Curated Data - Skinny
 
**Description** This notebook creates the collpased (selected) version of the skinny (KPC) table.

<br>Collapsed (one row per person) extracting the most appropriate KPC (Sex, Ethnicity, DOB) for the study. 

For this project we select the KPC **that is the most recent (in relation to the study end date)**, regardless of datasource.

In case of ties, we prioritise datasources as follows: HES APC, GDPPR, HES Outpatients, HES A&E, NACSA/TAVI.

Remaining KPC conflicts (different KPC values which have the same `RECORD_DATE` and the same data source) will be highlighted.
 
**Authors** Tom Bolton, Fionna Chalmers, Anna Stevenson (Health Data Science Team, BHF Data Science Centre)

**Reviewers** ⚠ UNREVIEWED

**Acknowledgements** Based on CCU002_07 and subsequently CCU003_05-D04-skinny

**Notes**

**Data Output**
- **`ccu056_tmp_kpc_harmonised`** : Skinny unassembled
- **`ccu056_tmp_skinny`** : Skinny assembled (selected)

# 0. Setup

In [0]:
spark.sql('CLEAR CACHE')
spark.conf.set('spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation', 'true')

In [0]:
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql import Window

from functools import reduce

import databricks.koalas as ks
import pandas as pd
import numpy as np

import re
import io
import datetime

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns

print("Matplotlib version: ", matplotlib.__version__)
print("Seaborn version: ", sns.__version__)
_datetimenow = datetime.datetime.now() # .strftime("%Y%m%d")
print(f"_datetimenow:  {_datetimenow}")

In [0]:
%run "/Repos//shds/common/functions"

In [0]:
%run "/Repos/shds/common/skinny_20221113"

# 1. Parameters

In [0]:
%run "./CCU056-01-parameters"

# 2. Data

In [0]:
gdppr   = extract_batch_from_archive(parameters_df_datasets, 'gdppr')
hes_apc = extract_batch_from_archive(parameters_df_datasets, 'hes_apc')
hes_ae  = extract_batch_from_archive(parameters_df_datasets, 'hes_ae')
hes_op  = extract_batch_from_archive(parameters_df_datasets, 'hes_op')

In [0]:
acs   = extract_batch_from_archive(parameters_df_datasets, 'nacsa')
tavi = extract_batch_from_archive(parameters_df_datasets, 'tavi')

In [0]:
# Main Cohort - needed only if the closest to procedure KPC used instead of the most recent
main_cohort = spark.table(f'{dsa}.ccu056_tmp_main_cohort_final')

In [0]:
all_unassembled = spark.table(f'{dsa}.{proj}_tmp_all_cases_unassembled')

In [0]:
display(all_unassembled)

# 4. Select KPCs

In [0]:
# kpc_selected = key_patient_characteristics_select(harmonised=kpc_harmonised)

# # temp save (~20 minutes)
# save_table(df=kpc_selected, out_name=f'{proj}_tmp_skinny', save_previous=True, data_base=dsa)

In [0]:
# remove those with missing ID, missing record date, and record date in the future  
all_unassembled = (
    all_unassembled
    .where(f.col('PERSON_ID').isNotNull())
    .where(f.col('RECORD_DATE').isNotNull())
    .where(f.col('RECORD_DATE') <= f.col('archived_on'))
    .withColumn('RECORD_SOURCE_PARENT',
                f.when(f.col('RECORD_SOURCE').isin(['nacsa', 'tavi']), "audits")
                .when(f.col('RECORD_SOURCE').isin(['gdppr']), "gdppr")
                .when(f.col('RECORD_SOURCE').isin(['hes_apc','hes_op','hes_ae']), "hes")
                .otherwise(f.lit(None))
    )
)

In [0]:
# in cases of ties when choosing closest to operate date - apply data priority groups
all_unassembled = (all_unassembled
        .withColumn('RECORD_SOURCE_group_final',
                    f.when(f.col('RECORD_SOURCE') == 'nacsa', 5)
                    .when(f.col('RECORD_SOURCE') == 'tavi', 5)
                    .when(f.col('RECORD_SOURCE') == 'gdppr', 3)
                    .when(f.col('RECORD_SOURCE') == 'gdppr_snomed', 4)
                    .when(f.col('RECORD_SOURCE') == 'hes_apc', 1)
                    .when(f.col('RECORD_SOURCE') == 'hes_op', 2)
                    .when(f.col('RECORD_SOURCE') == 'hes_ae', 2)
                    )
)

In [0]:
# right now study_end_date is pipeline production date but it will be changed to reflect the chosen study end date as at when
study_end_date

In [0]:
from datetime import datetime, timedelta

study_end_date_last = datetime.strptime(study_end_date, "%Y-%m-%d") - timedelta(days=1)
study_end_date_last = study_end_date_last.strftime("%Y-%m-%d")
study_end_date_last #this will be the last date considered so if RECORD_DATE on this then DATE_DIFF is 0

import datetime

In [0]:
# Filter for main cohort only - operation date is not needed for KPCS here
all_unassembled = (main_cohort.join(all_unassembled,on="PERSON_ID",how="left"))

In [0]:
display(all_unassembled)

In [0]:
# instead we need the study end date
all_unassembled = (all_unassembled     
.withColumn('STUDY_END_DATE',f.date_format(f.lit(study_end_date_last), "yyyy-MM-dd"))
#remove records after study end date
.where(f.col('RECORD_DATE') <= f.col('STUDY_END_DATE')) 
.withColumn('DATE_DIFF', f.abs(f.datediff(f.col("RECORD_DATE"), f.col("STUDY_END_DATE"))))
)

## KPC categories

In [0]:
ethnic_hes = spark.table(f'dss_corporate.hesf_ethnicity')
ethnic_gdppr = spark.table(f'dss_corporate.gdppr_ethnicity')
sex_gdppr = spark.table(f'dss_corporate.gdppr_sex')


**For Sex we want to prioritise records that are not null and Not known (null, 0. Not known, 0)**

**For ethnicity we want to prioritise records that are not null, blank Not Stated or Unknown, Not Known, Not Given (null, "", 9. Unknown, 99, 9, X,  Z)**

In [0]:
display(
    all_unassembled.select("RECORD_SOURCE_PARENT","SEX").distinct()
    .filter(f.col("RECORD_SOURCE_PARENT").isNotNull())
    .join((sex_gdppr.select(f.col("Value").alias("SEX"),f.col("Label").alias("SEX_DESC"))), on="SEX", how="left")
    .orderBy("RECORD_SOURCE_PARENT","SEX")
    .select("RECORD_SOURCE_PARENT","SEX","SEX_DESC")
)

In [0]:
ethnicity_codes = (
    all_unassembled.select("RECORD_SOURCE_PARENT","ETHNIC").distinct()
    .filter(f.col("RECORD_SOURCE_PARENT").isNotNull())
    .join((ethnic_gdppr.select(f.col("Value").alias("ETHNIC"),f.col("Label").alias("ETHNIC_DESC"))), on="ETHNIC", how="left")
    .withColumn("ETHNIC_DESC", f.when(f.col("RECORD_SOURCE_PARENT") != "gdppr", None).otherwise(f.col("ETHNIC_DESC")))
    .join((ethnic_hes.select(f.col("ETHNICITY_CODE").alias("ETHNIC"),f.col("ETHNICITY_DESCRIPTION").alias("ETHNIC_DESCRIPTION"))), on="ETHNIC", how="left")
    .withColumn("ETHNIC_DESCRIPTION", f.when(f.col("RECORD_SOURCE_PARENT") != "hes", None).otherwise(f.col("ETHNIC_DESCRIPTION")))
    .withColumn("ETHNIC_DESC", f.when(f.col("ETHNIC_DESCRIPTION").isNotNull(), f.col("ETHNIC_DESCRIPTION")).otherwise(f.col("ETHNIC_DESC")))
    .orderBy("RECORD_SOURCE_PARENT","ETHNIC")
    .select("RECORD_SOURCE_PARENT","ETHNIC","ETHNIC_DESC")
)

display(ethnicity_codes)

In [0]:
save_table(df=ethnicity_codes, out_name=f'{proj}_tmp_ethnicty_codes', save_previous=True, data_base=dsa)

In [0]:
_ethnic_mapping_snomed_primary = (
    spark.table('dss_corporate.gdppr_ethnicity_mappings')
)

display(_ethnic_mapping_snomed_primary)

## Priority Groups

In [0]:
all_unassembled = (
    all_unassembled
    .withColumn('DOB_null',
      f.when(
        (f.col('DOB').isNull())
        | (f.trim(f.col('DOB')).isin(['']))
        | (f.col('DOB') < '1900-01-01')
        | (f.col('DOB') > f.col('archived_on'))
        | (f.col('DOB') > f.col('RECORD_DATE'))
      , 1).otherwise(0)
    )
    .withColumn('SEX_null',
      f.when( (f.col('SEX').isNull()) | (f.trim(f.col('SEX')).isin(['', '0. Not known', '0'])), 1).otherwise(0)
    )
    .withColumn('ETHNIC_null',
      f.when( (f.col('ETHNIC').isNull()) | (f.trim(f.col('ETHNIC')).isin(['', '9', '9. Unknown', '99', 'X', 'Z'])), 1).otherwise(0)
    )
)


In [0]:
display(all_unassembled)

In [0]:
# check
display(all_unassembled.filter(f.col("SEX_null")==1).select("SEX").distinct())

In [0]:
# check
display(all_unassembled.filter(f.col("ETHNIC_null")==1).select("ETHNIC").distinct())

In [0]:
all_unassembled = (all_unassembled
        .withColumn('RECORD_SOURCE_DOB_group',
                    f.when(f.col('RECORD_SOURCE') == 'nacsa', 4)
                    .when(f.col('RECORD_SOURCE') == 'tavi', 4)
                    .when(f.col('RECORD_SOURCE') == 'gdppr', 2)
                    .when(f.col('RECORD_SOURCE') == 'gdppr_snomed', 3)
                    .when(f.col('RECORD_SOURCE') == 'hes_apc', 1)
                    .when(f.col('RECORD_SOURCE') == 'hes_op', 5)
                    .when(f.col('RECORD_SOURCE') == 'hes_ae', 5)
                    )
        .withColumn('RECORD_SOURCE_group',
                    f.when(f.col('RECORD_SOURCE') == 'nacsa', 5)
                    .when(f.col('RECORD_SOURCE') == 'tavi', 5)
                    .when(f.col('RECORD_SOURCE') == 'gdppr', 3)
                    .when(f.col('RECORD_SOURCE') == 'gdppr_snomed', 4)
                    .when(f.col('RECORD_SOURCE') == 'hes_apc', 1)
                    .when(f.col('RECORD_SOURCE') == 'hes_op', 2)
                    .when(f.col('RECORD_SOURCE') == 'hes_ae', 2)
                    )
        .withColumn('RECORD_SOURCE_group_final',
                    f.when(f.col('RECORD_SOURCE') == 'nacsa', 6)
                    .when(f.col('RECORD_SOURCE') == 'tavi', 6)
                    .when(f.col('RECORD_SOURCE') == 'gdppr', 4)
                    .when(f.col('RECORD_SOURCE') == 'gdppr_snomed', 5)
                    .when(f.col('RECORD_SOURCE') == 'hes_apc', 1)
                    .when(f.col('RECORD_SOURCE') == 'hes_op', 2)
                    .when(f.col('RECORD_SOURCE') == 'hes_ae', 3)
                    )
)

In [0]:
  # define windows for row numbers
  _win_rownum_DOB = (
    Window
    .partitionBy('PERSON_ID')\
    .orderBy(['DOB_null', 'DATE_DIFF', 'RECORD_SOURCE_DOB_group', 'RECORD_ID', 'RECORD_SOURCE_group_final']))
  _win_rownum_SEX = (
    Window
    .partitionBy('PERSON_ID')
    .orderBy(['SEX_null', 'DATE_DIFF', 'RECORD_SOURCE_group', 'RECORD_ID', 'RECORD_SOURCE_group_final']))
  _win_rownum_ETHNIC = (
    Window
    .partitionBy('PERSON_ID')
    .orderBy(['ETHNIC_null', 'DATE_DIFF', 'RECORD_SOURCE_group', 'RECORD_ID', 'RECORD_SOURCE_group_final']))
  
all_unassembled = (all_unassembled
    .withColumn('_rownum_DOB', f.row_number().over(_win_rownum_DOB))
    .withColumn('_rownum_SEX', f.row_number().over(_win_rownum_SEX))
    .withColumn('_rownum_ETHNIC', f.row_number().over(_win_rownum_ETHNIC))
    )

display(all_unassembled)


In [0]:

  
varlist = ['DOB', 'SEX', 'ETHNIC']
  
for ind, var in enumerate(varlist):
    record_source = 'RECORD_SOURCE_group'
    if(var == 'DOB'): record_source = 'RECORD_SOURCE_DOB_group'
    # define window for tied records
    _win_tie = (Window
      .partitionBy('PERSON_ID')
      .orderBy(f'{var}_null', record_source, 'DATE_DIFF')
      )
      
    # count distinct values of var (including null) within tied records
    _tie = (
      all_unassembled
      .withColumn(f'_tie_{var}', f.dense_rank().over(_win_tie))
      .where(f.col(f'_tie_{var}') == 1)
      .groupBy('PERSON_ID')
      .agg(
        f.countDistinct(f.col(f'{var}')).alias(f'_n_distinct_{var}')
        , f.countDistinct(f.when(f.col(f'{var}').isNull(), 1)).alias(f'_null_{var}')
      )
      .withColumn(f'_tie_{var}', f.when((f.col(f'_n_distinct_{var}') + f.col(f'_null_{var}')) > 1, 1).otherwise(0))
      .select('PERSON_ID', f'_tie_{var}'))
  
    if(ind == 0): _tmp_ties = _tie
    else: _tmp_ties = (_tmp_ties.join(_tie, on=['PERSON_ID'], how='outer'))

In [0]:
display(_tmp_ties)

In [0]:
  # take information from the first row identified above
_tmp_selected = {}
for var in varlist:
    _tmp = (
      all_unassembled
      .select('PERSON_ID', 'RECORD_DATE', 'RECORD_SOURCE', f'{var}', f'_rownum_{var}')
      .where(f.col(f'_rownum_{var}') == 1)
      .withColumnRenamed('RECORD_DATE', f'_date_{var}')
      .withColumnRenamed('RECORD_SOURCE', f'_source_{var}')
      .select('PERSON_ID', f'{var}', f'_date_{var}', f'_source_{var}'))
    _tmp_selected[f'{var}'] = _tmp

_selected = (
    _tmp_selected['DOB']
    .join(_tmp_selected['SEX'], on=['PERSON_ID'], how='outer')
    .join(_tmp_selected['ETHNIC'], on=['PERSON_ID'], how='outer')
    .join(_tmp_ties, on=['PERSON_ID'], how='outer')
    .select('PERSON_ID', 'DOB', 'SEX', 'ETHNIC'
            , '_date_DOB', '_source_DOB', '_tie_DOB'
            , '_date_SEX', '_source_SEX', '_tie_SEX'
            , '_date_ETHNIC', '_source_ETHNIC', '_tie_ETHNIC'))

In [0]:
display(_selected)

In [0]:
display(_selected.filter(f.col("PERSON_ID")=="7ND1SU07F76LLQZ"))

In [0]:
save_table(df=_selected, out_name=f'{proj}_tmp_all_cases_selected', save_previous=True, data_base=dsa)

In [0]:
display(_selected.groupBy("_tie_ETHNIC").count())

In [0]:
display(_selected.groupBy("_tie_SEX").count())

In [0]:
display(_selected.groupBy("_tie_DOB").count())

# 5. Sensitivity Analysis

Rerunning the above selection process for Sex as at the closest to proceudre date to access how many people have a different Sex value as at the date of operation to the most current.

In [0]:
all_unassembled_sex = (
    spark.table(f'{dsa}.{proj}_tmp_all_cases_unassembled')
    .drop("ETHNIC","CODE","DOB")
    
)

display(all_unassembled_sex)

In [0]:
# remove those with missing ID, missing record date, and record date in the future  
all_unassembled_sex = (
    all_unassembled_sex
    .where(f.col('PERSON_ID').isNotNull())
    .where(f.col('RECORD_DATE').isNotNull())
    .where(f.col('RECORD_DATE') <= f.col('archived_on'))
    .withColumn('RECORD_SOURCE_PARENT',
                f.when(f.col('RECORD_SOURCE').isin(['nacsa', 'tavi']), "audits")
                .when(f.col('RECORD_SOURCE').isin(['gdppr']), "gdppr")
                .when(f.col('RECORD_SOURCE').isin(['hes_apc','hes_op','hes_ae']), "hes")
                .otherwise(f.lit(None))
    )
)


# in cases of ties when choosing closest to operate date - apply data priority groups
all_unassembled_sex = (all_unassembled_sex
        .withColumn('RECORD_SOURCE_group_final',
                    f.when(f.col('RECORD_SOURCE') == 'nacsa', 5)
                    .when(f.col('RECORD_SOURCE') == 'tavi', 5)
                    .when(f.col('RECORD_SOURCE') == 'gdppr', 3)
                    .when(f.col('RECORD_SOURCE') == 'gdppr_snomed', 4)
                    .when(f.col('RECORD_SOURCE') == 'hes_apc', 1)
                    .when(f.col('RECORD_SOURCE') == 'hes_op', 2)
                    .when(f.col('RECORD_SOURCE') == 'hes_ae', 2)
                    )
)


# Filter for main cohort only - operation date is not needed for KPCS here
all_unassembled_sex = (main_cohort.join(all_unassembled_sex,on="PERSON_ID",how="left"))


# instead we need the study end date
all_unassembled_sex = (all_unassembled_sex
.withColumn('OPERATION_DATE',f.date_format(f.col("OPERATION_DATE"), "yyyy-MM-dd"))
.withColumn('DATE_DIFF', f.abs(f.datediff(f.col("RECORD_DATE"), f.col("OPERATION_DATE"))))
)



all_unassembled_sex = (
    all_unassembled_sex
    .withColumn('SEX_null',
      f.when( (f.col('SEX').isNull()) | (f.trim(f.col('SEX')).isin(['', '0. Not known', '0'])), 1).otherwise(0)
    )
)





all_unassembled_sex = (all_unassembled_sex

        .withColumn('RECORD_SOURCE_group',
                    f.when(f.col('RECORD_SOURCE') == 'nacsa', 5)
                    .when(f.col('RECORD_SOURCE') == 'tavi', 5)
                    .when(f.col('RECORD_SOURCE') == 'gdppr', 3)
                    .when(f.col('RECORD_SOURCE') == 'gdppr_snomed', 4)
                    .when(f.col('RECORD_SOURCE') == 'hes_apc', 1)
                    .when(f.col('RECORD_SOURCE') == 'hes_op', 2)
                    .when(f.col('RECORD_SOURCE') == 'hes_ae', 2)
                    )
        .withColumn('RECORD_SOURCE_group_final',
                    f.when(f.col('RECORD_SOURCE') == 'nacsa', 6)
                    .when(f.col('RECORD_SOURCE') == 'tavi', 6)
                    .when(f.col('RECORD_SOURCE') == 'gdppr', 4)
                    .when(f.col('RECORD_SOURCE') == 'gdppr_snomed', 5)
                    .when(f.col('RECORD_SOURCE') == 'hes_apc', 1)
                    .when(f.col('RECORD_SOURCE') == 'hes_op', 2)
                    .when(f.col('RECORD_SOURCE') == 'hes_ae', 3)
                    )
)




  # define windows for row numbers
_win_rownum_SEX = (
    Window
    .partitionBy('PERSON_ID')
    .orderBy(['SEX_null', 'RECORD_SOURCE_group', 'DATE_DIFF', 'RECORD_ID', 'RECORD_SOURCE_group_final']))

  
all_unassembled_sex = (all_unassembled_sex

    .withColumn('_rownum_SEX', f.row_number().over(_win_rownum_SEX))

    )

In [0]:
varlist = ['SEX']
  
for ind, var in enumerate(varlist):
    record_source = 'RECORD_SOURCE_group'
    # define window for tied records
    _win_tie = (Window
      .partitionBy('PERSON_ID')
      .orderBy(f'{var}_null', record_source, 'DATE_DIFF')
      )
      
    # count distinct values of var (including null) within tied records
    _tie = (
      all_unassembled_sex
      .withColumn(f'_tie_{var}', f.dense_rank().over(_win_tie))
      .where(f.col(f'_tie_{var}') == 1)
      .groupBy('PERSON_ID')
      .agg(
        f.countDistinct(f.col(f'{var}')).alias(f'_n_distinct_{var}')
        , f.countDistinct(f.when(f.col(f'{var}').isNull(), 1)).alias(f'_null_{var}')
      )
      .withColumn(f'_tie_{var}', f.when((f.col(f'_n_distinct_{var}') + f.col(f'_null_{var}')) > 1, 1).otherwise(0))
      .select('PERSON_ID', f'_tie_{var}'))
  
    if(ind == 0): _tmp_ties = _tie
    else: _tmp_ties = (_tmp_ties.join(_tie, on=['PERSON_ID'], how='outer'))



      # take information from the first row identified above
_tmp_selected = {}
for var in varlist:
    _tmp = (
      all_unassembled_sex
      .select('PERSON_ID', 'RECORD_DATE', 'RECORD_SOURCE', f'{var}', f'_rownum_{var}')
      .where(f.col(f'_rownum_{var}') == 1)
      .withColumnRenamed('RECORD_DATE', f'_date_{var}')
      .withColumnRenamed('RECORD_SOURCE', f'_source_{var}')
      .select('PERSON_ID', f'{var}', f'_date_{var}', f'_source_{var}'))
    _tmp_selected[f'{var}'] = _tmp

display(
    _tmp_selected['SEX'])

In [0]:
# _selected = (
#     _tmp_selected['SEX'].select('PERSON_ID',  'SEX', '_date_SEX', '_source_SEX', '_tie_SEX')
#     )

In [0]:
display(
    _tmp_selected['SEX'].groupBy("PERSON_ID").count().filter(f.col("count")>1)
    )

In [0]:
save_table(df=_tmp_selected['SEX'], out_name=f'{proj}_tmp_all_cases_selected_sex', save_previous=True, data_base=dsa)

In [0]:
display(_selected)